In [1]:
!pip install accelerate bitsandbytes peft datasets trl wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
# wandb.login()

In [1]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
len(dataset["train"])

3048

In [3]:
c = 0
for i in range(len(dataset["train"])):
#   print(dataset["train"][i]["label"])
  c += dataset["train"][i]["label"]

c, len(dataset["train"])

(1524, 3048)

In [4]:
def create_prompt(sample):

    prompt = '''
    Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.
    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.

    Q1 :- {Q1}
    Q2 :- {Q2}
    '''.format(Q1=sample["question_1"], Q2=sample["question_2"])

    prompt = "<s>[INST]" + prompt + "[/INST]"

    prompt += str(sample["label"]) + "</s>"


    return prompt

In [5]:
train_dataset = dataset["train"]

In [6]:
dataset_list = []

# create_prompt(train_dataset[4])

for i in range(len(train_dataset)):

    dataset_list.append([create_prompt(train_dataset[i]), train_dataset[i]["label"]])

In [7]:
import pandas as pd
from datasets import Dataset

dataset_df = pd.DataFrame(dataset_list, columns=["prompt", "label"])

dataset_new = Dataset.from_pandas(dataset_df)

In [8]:

# train_dataset =
train_dataset = dataset_new.select(range(100))
test_dataset = dataset_new.select(range(100,150))


In [9]:
# test_dataset
train_dataset[0]

{'prompt': '<s>[INST]\n    Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.\n    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.\n\n    Q1 :- After how many hour from drinking an antibiotic can I drink alcohol?\n    Q2 :- I have a party tonight and I took my last dose of Azithromycin this morning. Can I have a few drinks?\n    [/INST]1</s>',
 'label': 1}

In [10]:
import bitsandbytes
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer


nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float16
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
tokenizer.bos_token, tokenizer.eos_token

('<s>', '</s>')

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='cuda:0',
    quantization_config=nf4_config,
    use_cache=False,
    max_length = 1024
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
!nvidia-smi

Thu Mar 28 13:17:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0              30W /  70W |   4707MiB / 15360MiB |     24%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [16]:
prmpt = dataset_new[0]["prompt"]
prmpt = prmpt[:-5]
prmpt

'<s>[INST]\n    Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.\n    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.\n\n    Q1 :- After how many hour from drinking an antibiotic can I drink alcohol?\n    Q2 :- I have a party tonight and I took my last dose of Azithromycin this morning. Can I have a few drinks?\n    [/INST]'

In [17]:
# prmpt_enc = tokenizer.encode(prmpt)
# model.generate(torch.tensor(prmpt_enc))
device = "cuda:0"

encoding = tokenizer(prmpt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'[INST]\n    Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.\n    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.\n\n    Q1 :- After how many hour from drinking an antibiotic can I drink alcohol?\n    Q2 :- I have a party tonight and I took my last dose of Azithromycin this morning. Can I have a few drinks?\n    [/INST] 1'

In [18]:
prmpt = dataset_new[20]["prompt"]
prmpt = prmpt[:-5]
prmpt

'<s>[INST]\n    Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.\n    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.\n\n    Q1 :- Could homeopathy for acne help?\n    Q2 :- My doctor prescribed me Doxycycline for my pimples. How long would it take for it to work?\n    [/INST]'

In [19]:
# prmpt_enc = tokenizer.encode(prmpt)
# model.generate(torch.tensor(prmpt_enc))
device = "cuda:0"

encoding = tokenizer(prmpt, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        attention_mask = encoding.attention_mask,
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
response

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'[INST]\n    Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n    Given 2 medical questions Q1 and Q2, predict if both of them are similar or not.\n    If they are similar, output only 1 and nothing else. If not, then output only 0 and nothing else.\n\n    Q1 :- Could homeopathy for acne help?\n    Q2 :- My doctor prescribed me Doxycycline for my pimples. How long would it take for it to work?\n    [/INST] 0'

In [20]:
from peft import LoraModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [21]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [22]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "mistral_qlora_finetuning",
  #num_train_epochs=5,
  max_steps = 100,
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="steps",
#   evaluation_strategy="steps",
  evaluation_strategy="steps",
  eval_steps=20,
  learning_rate=2e-4,
  bf16=False,
  lr_scheduler_type='constant',
)

In [23]:
train_dataset, test_dataset

(Dataset({
     features: ['prompt', 'label'],
     num_rows: 100
 }),
 Dataset({
     features: ['prompt', 'label'],
     num_rows: 50
 }))

In [24]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  args=args,
  train_dataset=train_dataset,
  dataset_text_field = "prompt",
  eval_dataset=test_dataset
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:341: UserWarning: You passed `packing=True` to the SFTTrainer,

In [25]:
import time
start = time.time()
trainer.train()
print(time.time()- start)

wandb: Currently logged in as: jacobram90 (teammu). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.801200,0.704955
40,0.487500,0.663113
60,0.199900,0.844086
80,0.042000,1.050142
100,0.015900,1.191750


1244.7210021018982


In [26]:
test_dataset

Dataset({
    features: ['prompt', 'label'],
    num_rows: 50
})

In [27]:
import time


device = "cuda:0"


test_preds = []

s = time.time()
for batch in dataset_new.select(range(1500,1550)):

    prompt = batch["prompt"]
    prompt = prompt[:-5]
    # print(prompt)

    encoding = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    test_preds.append(response)
    # break

e = time.time()

print((e-s)/60, " mins")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

0.5446785926818848  mins


In [28]:
e-s

32.680715560913086

In [29]:
# test_preds[23]
cls_pred = []
for tp in test_preds:
  cls_pred.append(int(tp[-1]))

In [30]:
# test_dataset[0]["label"]
cls_true = []

for tp in dataset_new.select(range(1500,1550)):
  cls_true.append(tp["label"])

In [31]:
c = 0

for i in range(len(cls_true)):
  if cls_true[i] == cls_pred[i]:
    c += 1

print("Accuracy : ", 100*c/len(cls_true), "%")

Accuracy :  80.0 %
